In [ ]:
import numpy as np
import pandas as pd

from scipy.spatial.distance import jensenshannon
from scipy.stats import linregress, kstest

# Parameters

In [ ]:
DLL_COLUMNS = ['RichDLLe', 'RichDLLk', 'RichDLLmu', 'RichDLLp', 'RichDLLbt']
PARTICLE = 'pion'
output_dir = 'results'
run_date = '2024-dec-14'
output_dir_base = f'{output_dir}/{run_date}/layer'

# !unzip -qq '/content/drive/MyDrive/cern/data/results/30x30/dp_0.01/2024-oct-04/pion_sample_30x30.zip'
# y_sample = np.load('/content/results/pion_y_real.npy')
# x_sample = np.load('/content/results/pion_x_real.npy')
# t_generated = np.load('/content/results/t_generated.npy')

# Functions

In [ ]:
def estimate_distances(y_real, y_generated, uncertainty_scores, uncertainty_type = None, bin_type = 'linear',
                                                 particle_index = 0, metric='JS', n_rows = 2, n_cols = 5, dll_columns=DLL_COLUMNS):
  n_bins = n_rows * n_cols

  targets = np.array(y_real[:, particle_index])
  predictions = np.array(y_generated[:, particle_index])
  uncertainty_scores = np.array(uncertainty_scores)

  if uncertainty_type == 'MCD':
    uncertainty_scores = uncertainty_scores[:, particle_index]

  if bin_type == 'linear':
    bin_edges = np.linspace(uncertainty_scores.min(), uncertainty_scores.max(), n_bins + 1)
  else: # Quantiles
    bin_edges = np.quantile(uncertainty_scores, np.linspace(0, 1, n_bins + 1))

  # Digitize returns sample indices per bin
  bin_indices = np.digitize(uncertainty_scores, bin_edges)
  distances = []

  for i in range(10):
    indices = bin_indices == i + 1

    mins = targets[indices].min(), predictions[indices].min()
    maxs = targets[indices].max(), predictions[indices].max()

    hist_range = min(mins), max(maxs)

    targets_hist = np.histogram(targets[indices], 25, hist_range, True)[0]
    predictions_hist = np.histogram(
        predictions[indices], 25, hist_range, True)[0]

    if metric == 'JS':
      dist = jensenshannon(predictions_hist, targets_hist)
    else:
      dist = kstest(predictions[indices], targets[indices]).statistic

    distances += [dist]

  #print(f"{metric} Distances:\n" + ", ".join([str(dist) for dist in distances]))
  return bin_edges, distances


def estimate_correlation(all_bin_ranges, all_distances, dll_columns=DLL_COLUMNS):

  correlation_coefficient = []
  for i in range(5):
    bin_ranges = np.mean([all_bin_ranges[i][1:], all_bin_ranges[i][:-1]], 0)
    regress = linregress(bin_ranges, all_distances[i])
    correlation_coefficient += [regress.rvalue]
    #print(f'Correlation coefficient for {dll_columns[i]}:', regress.rvalue)

  return correlation_coefficient

In [ ]:
def calculate_stats(all_correlations, columns):
    df = pd.DataFrame(all_correlations, columns=columns)

    means = df.mean(axis=0)
    stds = df.std(axis=0)
    df.loc['Mean'] = means
    df.loc['Std'] = stds
    print(df)

In [ ]:
def calculate_correlations(metric, uncertainty_type, uncertainty_data, y_sample, t_generated, N = 30):
    all_correlations = []

    for j in range(N):
        all_bin_edges, all_distances = [], []
        for i in range(5):
            bin_edges, distances = estimate_distances(
                y_sample, t_generated, uncertainty_data[j],
                uncertainty_type=uncertainty_type, bin_type='quantiles',
                particle_index=i, metric=metric
            )

            all_bin_edges += [bin_edges]
            all_distances += [distances]

        all_correlations.append(estimate_correlation(all_bin_edges, all_distances))

    return all_correlations

# FD

## Load data

In [ ]:
layer = 14
dir = f'{output_dir_base}{layer}/'
fd_uncertainty_all = np.load(f'{dir}{PARTICLE}_fd_uncertainty.npy')
y_sample = np.load(f'{dir}{PARTICLE}_y_real.npy')
x_sample = np.load(f'{dir}{PARTICLE}_x_real.npy')
t_generated = np.load(f'{dir}{PARTICLE}_t_generated.npy')

## Features Densities with JS

In [ ]:
all_correlations = calculate_correlations('JS', 'FD', fd_uncertainty_all, y_sample, t_generated, len(fd_uncertainty_all))
calculate_stats(all_correlations, DLL_COLUMNS)

## Features Densities with KS

In [ ]:
all_correlations = calculate_correlations('KS', 'FD', fd_uncertainty_all, y_sample, t_generated, len(fd_uncertainty_all))
calculate_stats(all_correlations, DLL_COLUMNS)

# MCD

## Load data

In [ ]:
output_dir_base = f'{output_dir}/{run_date}/dp'
dp = 0.1
dir = f'{output_dir_base}{dp}/'

mcd_all_uncertainties  = np.load(f'{dir}{PARTICLE}_mcd_uncertainty.npy')
y_sample = np.load(f'{dir}{PARTICLE}_y_real.npy')
x_sample = np.load(f'{dir}{PARTICLE}_x_real.npy')
t_generated = np.load(f'{dir}{PARTICLE}_t_generated.npy')

## MCD with JS

In [13]:
all_correlations = calculate_correlations('JS', 'MCD', mcd_all_uncertainties, y_sample, t_generated, len(mcd_all_uncertainties))
calculate_stats(all_correlations, DLL_COLUMNS)

      RichDLLe  RichDLLk  RichDLLmu  RichDLLp  RichDLLbt
0     0.927535  0.943642  -0.096021  0.963068   0.912188
1     0.416817  0.943260  -0.646998  0.964932   0.947819
2     0.949274  0.949787  -0.336534  0.976682   0.915784
3     0.919528  0.958360   0.829548  0.985186   0.942625
4     0.957596  0.949527  -0.476767  0.976680   0.938662
5     0.564190  0.946825  -0.131832  0.965916   0.925347
6     0.925110  0.940391  -0.409269  0.966542   0.919202
7     0.887586  0.954094   0.707442  0.973679   0.937254
8     0.936102  0.942812  -0.720991  0.969330   0.920263
9     0.930097  0.957741   0.963019  0.969636   0.961393
10    0.944452  0.948556  -0.342156  0.975182   0.935092
11    0.957036  0.950576  -0.330050  0.978025   0.927810
12    0.956752  0.948657  -0.392965  0.966118   0.932744
13    0.895271  0.950104  -0.258945  0.974031   0.921510
14    0.925987  0.951511  -0.648492  0.987847   0.924302
15    0.917884  0.945129  -0.792083  0.972367   0.934306
16    0.924493  0.951809  -0.34

## MCD with KS

In [ ]:
all_correlations = calculate_correlations('KS', 'MCD', mcd_all_uncertainties)
calculate_stats(all_correlations, DLL_COLUMNS)